In [3]:
from loading_data import *

#### 1) Explain which classification problem you have chosen to solve. Is it a multi-class or binary classification problem?

The classification problem that we are solving is to predict chd (coronary heart disease) based on the attributes. It is a binary classification problem. The direct interpretation is to know if someone has had a chd based on data. We can extend this to predicting if someone will likely have a chd based on the collected data on this individual. (this is given that the person keeps that same habits) -> maybe this is not applicable because there is an age attribute.
Another use of this classification can be for an insurance company to determine if a client likely has had a chd to then be able to adjust the cost for the insurance.

#### 2. We will compare logistic regression, method 2 and a baseline. For logistic regression, we will once more use λ as a complexity-controlling parameter, and for method 2 a relevant complexity controlling parameter and range of values. We recommend this choice is made based on a trial run, which you do not need to report. Describe which parameter you have chosen and the possible values of the parameters you will examine. The baseline will be a model which compute the largest class on the training data, and predict everything in the test-data as belonging to that class (corresponding to the optimal prediction by a logistic regression model with a bias term and no features).

In [4]:
#First we want to normalize and transform our data.

from scipy.stats import zscore

normalised_X = np.copy(X)
#transform
normalised_X[:,6] = np.log(1 + X[:,6]) #add 1 because some alcohol values are 0
#normalise
normalised_X = zscore(normalised_X, axis = 0, ddof = 1)

attributeNames_norm = np.copy(attributeNames)
attributeNames_norm[6] = 'log-alc'
attributeNames_norm = ['normalized ' + attribute for attribute in attributeNames_norm]

#Or without the last binary data

Y = np.copy(normalised_X[:,:-1])
N_y, M_y = Y.shape

attributeNames_y = np.copy(attributeNames_norm[:-1])

In [5]:
import numpy as np
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
import torch
from sklearn.linear_model import LogisticRegression
from dtuimldmtools import train_neural_net

In [ ]:
#We need to consider the possible lambdas and the number of hidden units that we want to consider in the inner loop. 
#For the baseline model there is no controlling parameter

#First try
lambda_interval = np.logspace(-5, 2, 100)
n_hidden_units = [1,5,10,20]
# print(n_hidden_units)

K_out = 5
K_in = 5
CV_out = model_selection.KFold(K_out,shuffle=True)
CV_in = model_selection.KFold(K_in,shuffle=True)

# For statistical evaluation : store outer fold predictions for the three models
yhat = []
y_true = []

# For debugging : store inner fold predictions (for each outer fold)
Logistic_full = {}
ANN_full = {}
#where the last coordinate gives us if we are looking at the estimate or the validation values (0 for estimation and 1 for validation)

#The Error for the best model of each type of model in each outer loop
# Train_error = np.zeros((K_out,3))
Test_error= np.zeros((K_out,3))
best_lambda_index = np.int32(np.zeros(K_out))
best_h_index = np.int32(np.zeros(K_out))

for k, (train_index,test_index) in enumerate(CV_out.split(normalised_X,y)):
    print(f"#================OUTER LOOP {k+1}================#")
    #to store the new predictions of the selected model at each outer fold (to then be concatenated in yhat)
    dy = []

    #the training tests for each fold of the outer loop
    X_train = normalised_X[train_index, :]
    y_train = y[train_index]
    X_test = normalised_X[test_index, :]
    y_test = y[test_index]

    #Baseline model
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train,y_train)
    Test_error[k,2] = 1-baseline.score(X_test,y_test)
    dy.append(baseline.predict(X_test))

    #INITIALIZE ERROR HANDLING
    #Error for each model in each of the loops, overwritten at each outer loop 
    Logistic_Inner_test_error= np.zeros((K_in, len(lambda_interval)))
    ANN_Inner_test_error= np.zeros((K_in, len(n_hidden_units)))

    #average error of each model on each outter fold
    Logistic_Model_out_test_error = np.zeros((K_out, len(lambda_interval)))
    ANN_Model_out_test_error = np.zeros((K_out, len(n_hidden_units)))

    #we also need to store the sizes of the folds
    inner_fold_validate_sizes = np.zeros(K_in)

    #Inner Loop
    for i, (Inner_train_index, Inner_test_index) in enumerate(CV_in.split(X_train,y_train)):
        print(f"#================INNER LOOP {k+1}{i+1}================#")
        #initialize the training and validation sets
        X_subtrain = X_train[Inner_train_index]
        y_subtrain = y[Inner_train_index]
        X_validate = X_train[Inner_test_index]
        y_validate = y_train[Inner_test_index]

        #store the size of the validation set
        inner_fold_validate_sizes[i] = X_validate.shape[0]

        #=========#
        #Logistic regression
        #=========#

        #Logistic Regression Model Loop (hyper_parameter tuning)
        for s,lamb in enumerate(lambda_interval):
            print(f"#================Logistic Model {s+1}================#")
            
            mdl = LogisticRegression(penalty="l2", C=1 / lamb)
            mdl.fit(X_subtrain, y_subtrain)

            # y_subtrain_est = mdl.predict(X_train).T
            y_validate_est = mdl.predict(X_validate).T
            Logistic_Inner_test_error[i, s] = np.sum(y_validate_est != y_validate) / len(y_validate)
            Logistic_full[(i,s,k)] = { 'predictions' : y_validate_est, 'ground_truth' : y_validate}

        #==========#
        #ANN
        #==========#

        #We convert the training and test sets to torch tensors

        X_subtrain = torch.tensor(X_subtrain, dtype=torch.float32)
        y_subtrain = torch.tensor(y_subtrain, dtype=torch.float32).reshape(-1, 1)
        X_validate = torch.tensor(X_validate, dtype=torch.float32)
        y_validate = torch.tensor(y_validate, dtype=torch.float32).reshape(-1, 1)

        #ANN cross validation loop
        for j, n in enumerate(n_hidden_units):
            print(f"#================ANN Model {k+1}{i+1}{j+1}================#")
            # The lambda-syntax defines an anonymous function, which is used here to
            # make it easy to make new networks within each cross validation fold
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n),  # M features to H hiden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  
                # torch.nn.ReLU(),
                torch.nn.Linear(n, 1),  # H hidden units to 1 output neuron
                torch.nn.Sigmoid(),  # final tranfer function
            )
            #Loss function (Binary Cross Entropy)
            loss_fn = torch.nn.BCELoss()
            # Train for a maximum of 10000 steps, or until convergence
            max_iter = 10000
        
            net, final_loss, learning_curve = train_neural_net(
            model, loss_fn, X=X_subtrain, y=y_subtrain, n_replicates=3, max_iter=max_iter
            )
            y_validate_est = net(X_validate).detach().numpy()
            y_validate_pred = (y_validate_est > 0.5).astype(int)
            y_validate_np = y_validate.numpy()
            ANN_Inner_test_error[i, j] = np.sum(y_validate_pred != y_validate) / len(y_validate)
            ANN_full[(i,j,k)] = { 'predictions' : y_validate_pred.squeeze(), 'ground_truth' : y_validate_np.squeeze()}

    #Average Model Error calculation for Regression   
    Logistic_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*Logistic_Inner_test_error, axis = 0)/X_train.shape[0]
    best_lambda_index[k] = int(np.argmin(Logistic_Model_out_test_error))
    #Average Model Error calculation for ANN   
    ANN_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*ANN_Inner_test_error, axis = 0)/X_train.shape[0]
    best_h_index[k] = int(np.argmin(ANN_Model_out_test_error))

    #Retrain the best model on the full X_train for regression
    mdl = LogisticRegression(penalty="l2", C = 1/lambda_interval[best_lambda_index[k]])
    mdl.fit(X_train,y_train)
    y_test_est = mdl.predict(X_test).T
    Test_error[k,0] = np.sum(y_test_est != y_test) / len(y_test)
    
    #to store the predictions
    dy.append(y_test_est)

    #Retrain the best ANN model
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, n_hidden_units[best_h_index[k]]),  # M features to H hiden units
        # 1st transfer function, either Tanh or ReLU:
        # torch.nn.Tanh(),  
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden_units[best_h_index[k]], 1),  # H hidden units to 1 output neuron
        torch.nn.Sigmoid(),  # final tranfer function
    )
    #Loss function (Binary Cross Entropy)
    loss_fn = torch.nn.BCELoss()
    # Train for a maximum of 10000 steps, or until convergence
    max_iter = 10000
        
    net, final_loss, learning_curve = train_neural_net(
        model, loss_fn, X=X_train, y=y_train, n_replicates=3, max_iter=max_iter
    )
    y_test_est = net(X_test).detach().numpy()
    y_test_pred = (y_test_est > 0.5).astype(int)
    y_test = y_test.numpy()
    Test_error[k, 1] = np.sum(y_test_pred != y_test) / len(y_test)


    #to store the predictions
    dy.append(y_test_pred.squeeze())
    dy = np.stack(dy, axis=1)
    yhat.append(dy)
    y_true.append(y_test)

yhat = np.concatenate(yhat)
y_true = np.concatenate(y_true)

#================OUTER LOOP 1================#
#================INNER LOOP 11================#
#================Logistic Model 1================#
#================Logistic Model 2================#
#================Logistic Model 3================#
#================Logistic Model 4================#
#================Logistic Model 5================#
#================Logistic Model 6================#
#================Logistic Model 7================#
#================Logistic Model 8================#
#================Logistic Model 9================#
#================Logistic Model 10================#
#================Logistic Model 11================#
#================Logistic Model 12================#
#================Logistic Model 13================#
#================Logistic Model 14================#
#================Logistic Model 15================#
#================Logistic Model 16================#
#================Logistic Model 17================#
#================Logistic Mode

In [35]:
# Add the baseline predictions to the yhat, we can simulate that because the baseline is a proportion test, 
# therefore we can use the error to then simulate the predictions based on the y_validate


Here I make some python checks for myself

In [9]:
print(inner_fold_validate_sizes)
print(Logistic_Inner_test_error.shape)
print(Logistic_Inner_test_error[:,0])
print(sum(Logistic_Inner_test_error[:,0]))
print(np.sum(Logistic_Inner_test_error, axis = 0)[0])
#So the right axis to sum over is 0

[74. 74. 74. 74. 74.]
(5, 100)
[0.39189189 0.40540541 0.35135135 0.28378378 0.33783784]
1.7702702702702702
1.7702702702702702


In [10]:
print((inner_fold_validate_sizes[:,None] * Logistic_Inner_test_error)[:,0])
print(Logistic_Inner_test_error[:,0]*inner_fold_validate_sizes)
#So this broadcasting works as intended

[29. 30. 26. 21. 25.]
[29. 30. 26. 21. 25.]


In [11]:
print(ANN_Model_out_test_error)
print(Logistic_Model_out_test_error)

[0.01351351 0.01351351 0.01351351 0.01351351]
[0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405
 0.35405405 0.35405405 0.35405405 0.35405405 0.35405405 0.35675676
 0.35945946 0.35945946 0.35945946 0.35945946 0.35945946 0.35945946
 0.35945946 0.35945946 0.35945946 0.35945946 0.35945946 0.35675676
 0.35945946 0.35945946 0.35675676 0.35675676 0.35675676 0.35675676
 0.35675676 0.35675676 0.35945946 0.36216216 0.35675676 0.35945946
 0.35675676 0.35

In [39]:
# The lambda show that there should be no control parameter. 
# This means that the problem is inherently complicated and that there is not risk of overfitting with logistic regression.
# Maybe we can try with the logit link instead of the sigmoid function ?
print(f'optimal control parameter index : {best_lambda_index}')
print(f'optimal hidden unit parameter index : {best_h_index}')
print(f'optimal control parameter : {lambda_interval[best_lambda_index]}')
# print(f'Optimal number of hidden units : {n_hidden_units[np.int32(best_h_index)]}')
print(Test_error)

optimal control parameter index : [86 96 78  0 90]
optimal hidden unit parameter index : [0 0 0 0 0]
optimal control parameter : [1.20450354e+01 6.13590727e+01 3.27454916e+00 1.00000000e-05
 2.31012970e+01]
[[0.25806452 0.25806452 0.40860215]
 [0.2688172  0.29032258 0.27956989]
 [0.26086957 0.2826087  0.32608696]
 [0.34782609 0.36956522 0.39130435]
 [0.25       0.25       0.32608696]]


We can conclude that the logistic regression is the best fit here.

We go on with the rest and then come back on this issue afterwards

Perform a statistical evaluation of your three models similar to the previous section. That
is, compare the three models pairwise. We will once more allow some freedom in what test
to choose. Therefore, choose either:
setup I (section 11.3): Use McNemar’s test described in Box 11.3.2)
setup II (section 11.4): Use the method described in Box 11.4.1)
Include p-values and confidence intervals for the three pairwise tests in your report and
conclude on the results: Is one model better than the other? Are the two models better
than the baseline? Are some of the models identical? What recommendations would you
make based on what you’ve learned?

In [16]:
from dtuimldmtools import mcnemar

In [40]:
yhat = np.array(yhat)
y_true = np.array(y_true)

In [41]:
#we suppose that we have the y_hat and y_true for all models.

# Compute the Jeffreys interval
alpha = 0.05
[thetahatA, CIA, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 1], alpha=alpha)
[thetahatB, CIB, p] = mcnemar(y_true, yhat[:, 1], yhat[:, 2], alpha=alpha)
[thetahatA, CIB, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 2], alpha=alpha)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed